In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
import datetime
from datetime import timedelta 

# start_date_comments = datetime.date(2008, 12, 9) + timedelta(days=15)
# end_date_comments = datetime.date(2013, 11, 16)
# start_date_comments = datetime.date(2011, 12, 24) + timedelta(days=15)
# end_date_comments = datetime.date(2012, 3, 9)

# start_date_comments = datetime.date(2008, 12, 9)
# end_date_comments = datetime.date(2013,11, 16)

start_date_comments = datetime.date(2009, 12, 22)
end_date_comments = datetime.date(2013,10, 22)


In [ ]:
authors = pd.read_csv("Authors/authors.csv")
users_num = authors["id"].size

authors["zeros"] = 0.0
zeros = authors["zeros"]

In [ ]:
feature_table = pd.read_csv("FeatureTableCSV.csv", sep=";")

In [ ]:
def save_data_to_file(folder_name, file_name, data):
    try:
        # Create target Directory
        os.mkdir(folder_name)
        print("Directory " , folder_name ,  " Created ") 
    except FileExistsError:
        e = 1
        #print("Directory " , folder_name ,  " already exists")
    data.to_csv(folder_name + "/" + file_name, index=False)

In [ ]:
def merge_stats(start_date):
    statistics_to_join = []
    for index, row in feature_table.iterrows():
        folder = row[["NazwaFolderu","NazwaCechy"]][0]
        feature_name = row[["NazwaFolderu","NazwaCechy"]][1]
        statistics_to_join.append(pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")[['user_id',feature_name]])
    
    merged_df = statistics_to_join[0]
    for i in range(1, len(statistics_to_join)):
        merged_df = pd.merge(merged_df, statistics_to_join[i],how='outer',on=['user_id'])
    return merged_df.fillna(0)

In [ ]:
def create_cluster_data(start_date, end_date):
    while start_date < end_date:
        data = merge_stats(start_date)
        #res = pd.concat([res, data], ignore_index=True)
        print(str(start_date))
        #print(str(len(data)) + " " + str(start_date))
        save_data_to_file("All_Data_In_Slots_Joined", str(start_date) + "_joined.csv", data)
        start_date += timedelta(days=14)
    

In [ ]:
df = pd.read_csv("All_Data_In_Slots_Joined/" + str(start_date_comments) + "_joined.csv")
df

In [ ]:
#create_cluster_data(start_date_comments, end_date_comments)

In [ ]:
def generete_data_with_zeros(df, zeros, column_name):
    df = df[column_name]
    zeros.columns = column_name
    res = pd.concat([df,zeros[len(df):]])
    return res

In [ ]:
def standard_dev_feat_for_all_slots(folder, feature_name, start_date, end_date):
    res = pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")
    res_with_zeros = generete_data_with_zeros(res,zeros, feature_name)
    start_date += timedelta(days=14)
    while start_date < end_date:
        df = pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")
        data = generete_data_with_zeros(df, zeros, feature_name)
        #std = data.std()
        #result.append(std)
        res_with_zeros = pd.concat([res_with_zeros, data], ignore_index=True)
        start_date += timedelta(days=14)
    print(len(res_with_zeros.index))
    return res_with_zeros.std()

In [ ]:
def non_zero_standard_dev_feat_for_all_slots(folder, feature_name, start_date, end_date):
    res = pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")[feature_name]
    start_date += timedelta(days=14)
    while start_date < end_date:
        data = pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")[feature_name]
        res = pd.concat([res, data], ignore_index=True)
        #std = data[feature_name].std()
#         result.append(std)
        start_date += timedelta(days=14)
    print(len(res.index))
    return res.std()

In [ ]:
# def merge_stats(start_date):
#     statistics_to_join = []
#     for index, row in feature_table.iterrows():
#         folder = row[["NazwaFolderu","NazwaCechy"]][0]
#         feature_name = row[["NazwaFolderu","NazwaCechy"]][1]
#         statistics_to_join.append(pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")[['user_id',feature_name]])
    
#     merged_df = statistics_to_join[0]
#     for i in range(1, len(statistics_to_join)):
#         merged_df = pd.merge(merged_df, statistics_to_join[i],how='outer',on=['user_id'])
#     return merged_df.fillna(0)

In [ ]:
# def non_zero_feat_for_all_slots(folder, feature_name, start_date, end_date):
#     res = merge_stats(start_date)
#     start_date += timedelta(days=14)
#     while start_date < end_date:
#         data = merge_stats(start_date)
#         res = pd.concat([res, data], ignore_index=True)
#         start_date += timedelta(days=14)
#     num = len(res)
#     return (num / users_num) * 100

In [ ]:
for index, row in feature_table.iterrows():
    folder = row[["NazwaFolderu","NazwaCechy"]][0]
    feature_name = row[["NazwaFolderu","NazwaCechy"]][1]
    if(folder == "NumberOfReceivedResponsesUnderComments"):
        print("TYPE       | " + feature_name )
        print("STD ZERO   | " + str(standard_dev_feat_for_all_slots(folder, feature_name, start_date_comments, end_date_comments)))
        print("STD NONZERO| " + str(non_zero_standard_dev_feat_for_all_slots(folder, feature_name, start_date_comments, end_date_comments)))
#     print("NONZERO    | " + str(non_zero_feat_for_all_slots(folder, feature_name, start_date_comments, end_date_comments)))

In [ ]:
for index, row in feature_table.iterrows():
    folder = row[["NazwaFolderu","NazwaCechy"]][0]
    feature_name = row[["NazwaFolderu","NazwaCechy"]][1]
    print("TYPE       | " + feature_name )
    print("STD ZERO   | " + str(standard_dev_feat_for_all_slots(folder, feature_name, start_date_comments, end_date_comments)))
    print("STD NONZERO| " + str(non_zero_standard_dev_feat_for_all_slots(folder, feature_name, start_date_comments, end_date_comments)))
#     print("NONZERO    | " + str(non_zero_feat_for_all_slots(folder, feature_name, start_date_comments, end_date_comments)))

In [ ]:
def plot_standard_dev_feat_over_months(file_name, feature_name, start_date, end_date):
    result = []
    while start_date < end_date:
        df = pd.read_csv(file_name + "/feature_" + str(start_date) + ".csv")
        data = generete_data_with_zeros(df,zeros, feature_name)
        std = data.std()
        result.append(std)
        start_date += timedelta(days=14)
    plt.plot(result)
    plt.ylabel('STD value of {}'.format(feature_name))
    plt.savefig("FeatureAnalysisFigs/" + "STD_" + feature_name)
    plt.show()

In [ ]:
def plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date, end_date):
    result = []
    while start_date < end_date:
        data = pd.read_csv(file_name + "/feature_" + str(start_date) + ".csv")
        std = data[feature_name].std()
        result.append(std)
        start_date += timedelta(days=14)

    plt.plot(result)
    plt.ylabel('NON ZERO STD value of {}'.format(feature_name))
    plt.savefig("FeatureAnalysisFigs/" + "NONZERO_STD_" + feature_name)
    plt.show()

In [ ]:
def plot_non_zero_feat_over_months(file_name, feature_name, start_date, end_date):
    result = []
    while start_date < end_date:
        data = pd.read_csv(file_name + "/feature_" + str(start_date) + ".csv")
        without_zeros = data.loc[data[feature_name] != 0.0]
        num = len(without_zeros)
        prec = (num / users_num) * 100
        result.append(prec)
        start_date += timedelta(days=14)
    plt.plot(result)
    plt.ylabel('Percentage of non zero value of {}'.format(feature_name))
    plt.savefig("FeatureAnalysisFigs/" + "PROCENTAGE_" + feature_name)
    plt.show()

In [ ]:
# feature_table = pd.read_csv("FeatureTableCSV.csv", sep=";")
# start_date = start_date_comments
# end_date = end_date_comments

In [ ]:
# folder = "FrequencyOfPosts"
# feature_name = list(feature_table.loc[feature_table["NazwaFolderu"] == "FrequencyOfPosts"]["NazwaCechy"])[0]

In [ ]:
def plot_histogram(folder, feature_name, start_date, end_date):
    
    res = pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")[feature_name]
    start_date += timedelta(days=14)
    while start_date < end_date:
        df2 = pd.read_csv(folder + "/feature_" + str(start_date) + ".csv")[feature_name]
        res = pd.concat([res, df2], ignore_index=True)
        start_date += timedelta(days=14)

    plt.gcf().clear()
#     binwidth = max(res) / 100
#     bins=range(min(res), max(res) + 20, 20)
#     plt.gca().set_xlim([0, max(res)])
    plt.yscale('log', nonposy='clip')
    plt.hist(res.T, label=feature_name, histtype='bar', bins=100, alpha=0.5, edgecolor='black', facecolor='blue')
    plt.legend()
    plt.savefig("FeatureAnalysisFigs/" + "HIST_" + feature_name)
    plt.show()

# New Feature Names

In [ ]:
feature_table = pd.read_csv("FeatureTableCSV.csv", sep=";")

In [ ]:
for index, row in feature_table.iterrows():
    folder = row[["NazwaFolderu","NazwaCechy"]][0]
    feature_name = row[["NazwaFolderu","NazwaCechy"]][1]
    
    plot_non_zero_feat_over_months(folder, feature_name, start_date_comments, end_date_comments)
    plot_non_zero_standard_dev_feat_over_months(folder, feature_name, start_date_comments, end_date_comments)
    plot_standard_dev_feat_over_months(folder, feature_name, start_date_comments, end_date_comments)
    plot_histogram(folder, feature_name, start_date_comments, end_date_comments)

 ### FrequencyOfUserComments

In [ ]:
file_name = "FrequencyOfUserComments"

In [ ]:
feature_name = "mean_comments_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "std_comments_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "median_comments_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "min_comments_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "max_comments_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "q3_comments_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### FrequencyOfUserPosts

In [ ]:
file_name = "FrequencyOfUserPosts"

In [ ]:
feature_name = "mean_post_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "std_post_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "median_post_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "min_post_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "max_post_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "q3_post_frequency"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfCommentsWrittenByUserUnderHisOwnPostsWithoutZeros

In [ ]:
file_name = "NumberOfCommentsWrittenByUserUnderHisOwnPostsWithoutZeros"

In [ ]:
feature_name = "number_of_comments_written_by_user_under_his_own_posts_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_comments_written_by_user_under_his_own_posts_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_comments_written_by_user_under_his_own_posts_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_comments_written_by_user_under_his_own_posts_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_comments_written_by_user_under_his_own_posts_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_comments_written_by_user_under_his_own_posts_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfReceivedResponsesToUsersPostsWithoutZeros

In [ ]:
file_name = "NumberOfReceivedResponsesToUsersPostsWithoutZeros"
#features = ["number_of_received_responses_to_users_posts_avg","number_of_received_responses_to_users_posts_std","number_of_received_responses_to_users_posts_median","number_of_received_responses_to_users_posts_q3","number_of_received_responses_to_users_posts_max","number_of_received_responses_to_users_posts_min"]

In [ ]:
feature_name = "number_of_received_responses_to_users_posts_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_to_users_posts_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_to_users_posts_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_to_users_posts_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_to_users_posts_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_to_users_posts_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfReceivedResponsesUnderUsersCommentsWithoutZeros

In [ ]:
file_name = "NumberOfReceivedResponsesUnderUsersCommentsWithoutZeros"

In [ ]:
feature_name = "number_of_received_responses_under_users_comments_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_under_users_comments_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_under_users_comments_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_under_users_comments_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_under_users_comments_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_received_responses_under_users_comments_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfUsersPostsWithoutZeros

In [ ]:
file_name = "NumberOfUsersPostsWithoutZeros"

In [ ]:
feature_name = "number_of_posts"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfWordsInOwnResponsesOfUsersPostsWithoutZeros

In [ ]:
file_name = "NumberOfWordsInOwnResponsesOfUsersPostsWithoutZeros"

In [ ]:
feature_name = "number_of_words_in_own_responses_of_users_posts_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_own_responses_of_users_posts_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "nnumber_of_words_in_own_responses_of_users_posts_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_own_responses_of_users_posts_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_own_responses_of_users_posts_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_own_responses_of_users_posts_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfWordsInResponsesOfUsersPostsWithoutZeros

In [ ]:
file_name = "NumberOfWordsInResponsesOfUsersPostsWithoutZeros"

In [ ]:
feature_name = "number_of_words_in_responses_of_users_posts_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_responses_of_users_posts_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "nnumber_of_words_in_responses_of_users_posts_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_responses_of_users_posts_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_responses_of_users_posts_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_responses_of_users_posts_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfWordsInUsersCommentsWithoutZeros

In [ ]:
file_name = "NumberOfWordsInUsersCommentsWithoutZeros"

In [ ]:
feature_name = "number_of_words_in_users_comments_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_comments_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_comments_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_comments_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_comments_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_comments_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfWordsInUsersPostsWithoutZeros

In [ ]:
file_name = "NumberOfWordsInUsersPostsWithoutZeros"

In [ ]:
feature_name = "number_of_words_in_users_posts_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_posts_std"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_posts_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_posts_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_posts_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "number_of_words_in_users_posts_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### NumberOfWrittenCommentsToOtherUsersPostsWithoutZeros

In [ ]:
file_name = "NumberOfWrittenCommentsToOtherUsersPostsWithoutZeros"

In [ ]:
feature_name = "number_of_written_comments_to_other_users_posts"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### SentimentOfUsersCommentsWithoutZeros

In [ ]:
file_name = "SentimentOfUsersCommentsWithoutZeros"

In [ ]:
feature_name = "comments_sentiment_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "comments_sentiment_stddev"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "comments_sentiment_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "comments_sentiment_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "comments_sentiment_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "comments_sentiment_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### SentimentOfUsersPostsWithoutZeros

In [ ]:
file_name = "SentimentOfUsersPostsWithoutZeros"

In [ ]:
feature_name = "posts_sentiment_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "posts_sentiment_stddev"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "posts_sentiment_median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "posts_sentiment_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "posts_sentiment_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "posts_sentiment_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

### SentimentOfUsersResponsesToUserPostsWithoutZeros

In [ ]:
file_name = "SentimentOfUsersResponsesToUserPostsWithoutZeros"

In [ ]:
feature_name = "received_comments_sentiment_avg"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "received_comments_sentiment_stddev"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "received_comments_sentiment__median"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "received_comments_sentiment_q3"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "received_comments_sentiment_max"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)

In [ ]:
feature_name = "received_comments_sentiment_min"
plot_non_zero_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_non_zero_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)
plot_standard_dev_feat_over_months(file_name, feature_name, start_date_comments, end_date_comments)